In [444]:
import pandas as pd
import os
from google.colab import drive
import shutil
from sklearn.model_selection import train_test_split
import reorganize_extracted_data_api
# import importlib
from reorganize_extracted_data_api import *
# importlib.reload(reorganize_extracted_data_api)

# This code was used to reorganize the data in our Google Drive

# Original Organization Format

After extracting all the audio files, they were in the following format:

Extracted_data (root)

├── 20200505 (dir giving date)

│   ├── 0Ha52POVIxTKEPqI1eGpIoMHUd52 (dir giving ID)

│   │   └── breathing-deep.wav (audio file)

│   │   └── breathing-shallow.wav (audio file)

│   │   └── cough-heavy.wav (audio file)

│   │   └── *********.wav etc. (audio files)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3 (dir giving ID)

│   │   └── cough-heavy.wav (audio file)

│   │   └── *********.wav etc. (audio files)

│   ├── ************************* etc. (dir giving ID)

├── 20200525 (dir)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3 (dir giving ID)

│   │   └── cough-heavy.wav (audio file)

│   │   └── *********.wav etc. (audio files)

│   ├── ************************* etc. (dir giving ID)

└── ######## etc. (dir)

# New Organization Format

This code then reorganizes our files into separate folders, one for each of the sounds. The only contents of these folders are the audio files which have been edited to include the ID in their names

Extracted_data (root)

├──  breathing-deep (dir giving type of audio)

│   ├── 0Ha52POVIxTKEPqI1eGpIoMHUd52_breathing-deep.wav (audio file w/ID)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3_breathing-deep.wav (audio file w/ID)

│   │   └── *********_******.wav etc. (audio files w/IDs)

├──  breathing-shallow (dir giving type of audio)

│   ├── 0Ha52POVIxTKEPqI1eGpIoMHUd52_breathing-shallow.wav (audio file w/ID)

│   ├── 0HIgO2EhOOW1msCbEw1kC8Qsx6D3_breathing-shallow.wav (audio file w/ID)

│   │   └── *********_******.wav etc. (audio files w/IDs)

├──  ***** etc. (dir giving type of audio)

│   │   └── *********_******.wav etc. (audio files w/IDs)


In [445]:
# Mount Google Drive
drive.mount("/content/gdrive", force_remount=True) # Run and choose your school google drive and copy and paste the key and press enter

Mounted at /content/gdrive


In [446]:
# Set path you want to clean up and path to move to
# path = '/content/gdrive/MyDrive/dumbstuff'
# new_path = '/content/gdrive/MyDrive/dumbstuff2'
# path = '/content/gdrive/MyDrive/Extracted_data_3'
# new_path = '/content/gdrive/MyDrive/Extracted_data_4'
# path = '/content/gdrive/MyDrive/abc1'
# new_path = '/content/gdrive/MyDrive/abc2'
path = '/content/gdrive/MyDrive/DSCI400/Coswara-Data-master/Coswara-Data-master/Extracted_data'
new_path = '/content/gdrive/MyDrive/DSCI400/Reorganized Audio 2'

In [447]:
# Create new directory
make_new_dir(new_path)

# Set working directory
os.chdir(path) 

desired_directories = ['breathing-deep', 'breathing-shallow', 'cough-heavy', 'cough-shallow', 'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o', 'metadata']
# desired_directories = ['txt1', 'txt2']

In [448]:
import pandas as pd
import os
from google.colab import drive
import shutil
from sklearn.model_selection import train_test_split
from os import listdir
from os.path import isfile, join

def make_new_dir(new_path):
  if not os.path.exists(new_path):
      os.makedirs(new_path)
  else:
      shutil.rmtree(new_path)           
      os.makedirs(new_path)

def subfolder_extractor(source, target, operation, flag):
  """
    Performs extraction of subfolders within a specified parent folder.
    It can either simply remove the first layer of subfolders or can 
    do so in addition to renaming the files with the names of their respective
    parent folders.

    inputs:
        - source: the subfolder one is attempting to remove
        - target: the target folder to extract the desired files and/or
                  directories to
        - operation: move or copy the files
        - flag: 0 represents to simply remove the first layer of subfolders
                1 represents to remove the first layer of subfolders as well as
                  to rename the files to include the names of their parent
                  folders

    Returns: N/A
    """
  # Define root and target directories 
  root_src_dir = os.path.join('.', source)
  root_target_dir = os.path.join('.',target)

  if (flag):
    depth = 1
  else:
    depth = 2

  # print('root_src_dir: ' + str(root_src_dir))
  # print('root_target_dir: ' + str(root_target_dir))
  # print('-------------------------')
  # Walk across all files and directories in our root directory

  for src_dir, dirs, files in os.walk(root_src_dir):
    if src_dir[len(root_src_dir):].count(os.sep) < depth:
  
  #for src_dir, dirs, files in os.walk(root_src_dir):
      # print('src_dir: ' + src_dir)
      # print('dirs: ' + str(dirs))
      # print('files: ' + str(files))
      # print('-----------------------')
      dst_dir = src_dir.replace(root_src_dir, root_target_dir)
      # print('dst_dir: ' + dst_dir)
      # print('root_src_dir: ' + root_src_dir)
      # print('root_target_dir: ' + root_target_dir)
      # print('=======================')
    # Create the necessary destination directories to move the copies of the
    #   contents of the subfolders one is trying to remove.
      if not os.path.exists(dst_dir):
        os.mkdir(dst_dir)

      for file_ in files:
      # If flag == 1, then rename files and remove their parent folders
        if (flag):
          ending = os.path.basename(os.path.normpath(root_src_dir))
          os.replace(os.path.join(root_src_dir, file_), os.path.join(dst_dir, ending + '_' + file_))

      # If flag == 0, then simply remove the subfolders of the source directory
      #   and move the files contained in the subfolders to the src directory 
        else:  
          src_file = os.path.join(src_dir, file_)
          dst_file = os.path.join(dst_dir, file_)
          if os.path.exists(dst_file):
            os.remove(dst_file)

        # Either copy or move the remaining files in the original subfolders
          if operation is 'copy':
            shutil.copy(src_file, dst_dir)
          elif operation is 'move':
            shutil.move(src_file, dst_dir)

def cleaner(data_dir, new_path, desired_directories, flag):
  """
    Calls subfolder_extractor function and cleans up any remaining empty
    subfolders within the source folder given by data_dir

    inputs:
        - data_dir: the source folder one is attempting to clean
        - new_path: the new destination directory to move to
        - flag: 0 represents to simply remove the first layer of subfolders
                1 represents to remove the first layer of subfolders as well as
                  to rename the files to include the names of their parent
                  folders

    Returns: N/A
    """
  # List comprehension to compile all the directories within data_dir
  directories = [d for d in os.listdir(data_dir) 
                 if os.path.isdir(os.path.join(data_dir, d))]

  # Compile a list containing the full filepaths of the subdirectories of data_dir
  dir_paths = list()
  for i in directories:
    dir_paths.append(os.path.join(data_dir, i))
  
  if (flag):
    operation = 'move'
  else:
    operation = 'copy'
    new_dirs = [d for d in os.listdir(new_path) 
                 if os.path.isdir(os.path.join(new_path, d))]
    for i in desired_directories:
      if i in new_dirs:
        shutil.rmtree(os.path.join(new_path, i))
    
  # Call the subfolder_extractor function for all subfolders of data_dir
  for i in dir_paths:
    subfolder_extractor(i, new_path, operation, flag)

  # Remove all the original subfolders since we have moved their contents into
  #   the folder defined by data_dir
  if (flag):
    for i in directories:
      shutil.rmtree(os.path.join(new_path, i))

def org_into_sounds(data_dir, desired_directories):
  """
    Organizes the files in the source directory into their appropriate
      subdirectories within the source directory

    inputs:
        - data_dir: the source folder one is attempting to organize
        - desired_directories: a list of desired directories that the user
                               intends to sort all files in data_dir into

    Returns: N/A
    """
  # Creates directories in data_dir as listed in desired_directories
  for d in desired_directories:
    make_new_dir(os.path.join(data_dir, d))

  # Copies files from data_dir into their appropriate folders
  only_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
  for file_ in only_files:
    for i in desired_directories:
      if (i in file_):
        shutil.copy(file_, os.path.join(i, file_))
  
  # Removes all files from data_dir since all files have been copied into
  #   their appropriate folders

  all_files = []
  for (dirpath, dirnames, filenames) in os.walk(data_dir):
    all_files.extend(filenames)
    break
  for i in all_files:
    os.remove(i)

In [ ]:
# The first function call will simply remove the first layer of directories
#   beneath the root folder as defined in the path variable. It does not edit
#   any of the names or contents of the subfolders.
cleaner(path, new_path, desired_directories, 0)

In [ ]:
# The second function call will rename the files in each remaining subfolder
#   in the root folder to include the name of the subfolder, followed by an
#   underscore, followed by the original name of the file. It then sorts the
#   renamed file into a folder which only contains files of its corresponding
#   sound. The "1" is used as a flag to represent this case.
cleaner(new_path, new_path, desired_directories, 1)

In [ ]:
os.chdir(new_path) 
org_into_sounds(new_path, desired_directories)